In [1]:
import pandas as pd 
import numpy as np
import itertools 
import requests
import ortools
import os

In [2]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [3]:
nx = pd.read_excel('南洲镇网格站覆盖门店信息_1202(1).xlsx',dtype={'StoreCode':str})

In [4]:
nx.head()

,Unnamed: 0,StoreCode,SName,Longitude,Latitude,d_avg,directorId,directorName,endDeliveryTime
0,NaN,0,0,112.378082,29.357763,0,0,0,2020-12-02 00:00:00
1,0.0,00055878,南县南洲李灿超市,112.400640,29.363728,42,23822,H_祝有益,2020-11-28 08:15:58
2,1.0,00053326,南县南洲佳平便利店,112.406280,29.368052,114,23821,F_熊孟良,2020-11-28 10:26:10
3,2.0,50050895,南县南洲洪福超市,112.399520,29.360931,60,23822,H_祝有益,2020-11-28 09:19:34
4,3.0,00055817,南县南洲兆廷食品店,112.425010,29.363157,92,23982,G_杨程,2020-11-28 08:08:52


In [7]:
nx.head()

,Unnamed: 0,StoreCode,SName,Longitude,Latitude,d_avg,directorId,directorName,endDeliveryTime,capacity
0,NaN,0,0,112.378082,29.357763,0,0,0,2020-12-02 00:00:00,850
1,0.0,00055878,南县南洲李灿超市,112.400640,29.363728,42,23822,H_祝有益,2020-11-28 08:15:58,850
2,1.0,00053326,南县南洲佳平便利店,112.406280,29.368052,114,23821,F_熊孟良,2020-11-28 10:26:10,850
3,2.0,50050895,南县南洲洪福超市,112.399520,29.360931,60,23822,H_祝有益,2020-11-28 09:19:34,850
4,3.0,00055817,南县南洲兆廷食品店,112.425010,29.363157,92,23982,G_杨程,2020-11-28 08:08:52,850


In [5]:
nx.dtypes

Unnamed: 0                float64
StoreCode                  object
SName                      object
Longitude                 float64
Latitude                  float64
d_avg                       int64
directorId                  int64
directorName               object
endDeliveryTime    datetime64[ns]
dtype: object

In [ ]:
#高德地图调取导航距离
def get_distance(ox,oy,tx,ty):
    # https://restapi.amap.com/v3/direction/driving?parameters
    k = 'c33b02b68bd7785d75ed05f0178a8e05'
    if ox == 0 or oy == 0 or str(tx).lower().strip() == 'nan'
    origin = str(ox) + ',' + str(oy)
    destination = str(tx) + ',' + str(ty)
    url = "https://restapi.amap.com/v3/direction/driving?key=%s&origin=%s&destination=%s&strategy=2&extensions=base" % (
        k, origin, destination)
    headers = {'connection': 'close',}
    try:
        data = request.get(url,headers = headers)
        contest = data.json
        if contest['status'] == '1'
            route = contest['route']
            result = route['path'][0]
            out = float(result['distance'])/1000
        else:
            print(contest['info'])
            print(ty) 
            out = 999
        
    except:
        out = 999
        
    return out

In [ ]:
#计算每个门店到彼此间的距离
def cal_distance_for_each (cal_df):
    requests.adapters.DEFAULT_RETRIES = 5
    s = requests.Session()
    s.keep_alive = False
    cal_df['navi_distance'] = cal_df.apply(lambda df: get_distance(df['lon0'],df['lat0'],df['lon1'],df['lat1']),axis = 1)
    return cal_df.copy()  

                                                                                                                                                                                                                                                                

In [ ]:
#创建遍历list
lnglat = nx.value_to_list()
dist_combination = list(itertools.combinations(lnglat,2))


#路径规划排序
order_list = []
for i in dist_combination:
    temp_list = []
    index = ''
    if str(i[0][0]) > str(i[1][0]):
        index = str(i[1][0]) + '-' + str(i[0][0])
        temp_list.extend(i[1]+i[0])
    else:
        index = str(i[0][0]) + '-' + str(i[1][0])
        temp_list.extend(i[0]+i[1])
    temp_list.append(index)
    order_list.append(temp_list)
    
#apply 路径规划排序
cal_combination_df = pd.DataFrame(data = order_list,columns =['village_code_0','lon0','lat0',
                                  'village_code_1','lon1','lat1','village_distance_index'])
#编码转换
cal_combination_df['village_code_0'] = cal_combination_df['village_code_0'].astype(str)
cal_combination_df['village_code_1'] = cal_combination_df['village_code_1'].astype(str)

In [8]:
 [850]*40

[850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850,
 850]

In [ ]:
# define demands & vapacities  
data['demands'] = cal_combination_df['d_avg']
data['vehicle_capacities'] = [850]*40
#路径规划函数
def get_route_total_distance(store_table, store_distance_df):
  manager = pywrapcp.RoutingIndexManager(store_table.shape[0], 18, 0)
  routing = pywrapcp.RoutingModel(manager)  
   
    #  创建distance_callback函数
    def distance_callback(from_index,to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        if to_node == 0
            return 0
        from_code = store_table.loc[int(from_node),'门店编号']
        to_code = store_table.loc[int(to_node),'门店编号']
        
        if from_code > to_code
            return store_distance_df[(store_distance_df['village_code_1'] ==from_node) & (store_distance_df['village_code_0'] == to_node)].values[0]
        if from_code < to_code
            return store_distance_df[(store_distance_df['village_code_0'] == from_code) & (store_distance_df['village_code_1'] == to_code)].values[0]
        
        else:
            return 0 
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
      # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
     
   # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')
  
  # Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)



